In [373]:
import requests
from bs4 import BeautifulSoup
from itertools import cycle
import pandas as pd
import re

In [ ]:
# show all columns in the df
pd.set_option('display.max_columns', None)

In [377]:
links_list = []  # Create a new list to store the links

In [379]:
# scrape all links from a car brand
def get_links(car_brand):
    page = 1
    while page <= 13:
        url = f"https://www.bilbasen.dk/brugt/bil/{car_brand}?zipcode=0000&includeengroscvr=true&includesellforcustomer=true&includewithoutvehicleregistrationtax=true&includeleasing=false&includecallforprice=true&hpfrom=&hpto=&page=" + str(page)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        links = soup.find_all("a", {"class": "listing-heading"})
        for link in links:
            links_list.append(link["href"])  # Append the link to the new list

        page += 1

get_links(car_brand="dacia")

In [380]:
# convert the list into dataframe and add https://www.bilbasen.dk to the text link
data = pd.DataFrame(links_list)
data.columns = ['link']
for d in data:
    data = "https://www.bilbasen.dk"+ data[d]

In [381]:
data

0      https://www.bilbasen.dk/brugt/bil/dacia/logan/...
1      https://www.bilbasen.dk/brugt/bil/dacia/logan/...
2      https://www.bilbasen.dk/brugt/bil/dacia/spring...
3      https://www.bilbasen.dk/brugt/bil/dacia/jogger...
4      https://www.bilbasen.dk/brugt/bil/dacia/sander...
                             ...                        
449    https://www.bilbasen.dk/brugt/bil/dacia/logan/...
450    https://www.bilbasen.dk/brugt/bil/dacia/sander...
451    https://www.bilbasen.dk/brugt/bil/dacia/sander...
452    https://www.bilbasen.dk/brugt/bil/dacia/duster...
453    https://www.bilbasen.dk/brugt/bil/dacia/lodgy/...
Name: link, Length: 454, dtype: object

In [384]:
# car list
cars = [] 
# Udtruk og manipuler data
def scraping_data():

    
    for list in data:

        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

        # URL til at scrappe
        urls = list
        response = requests.get(urls, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")

        
        brands = soup.find_all("a", {"class": "bas-MuiTypography-root bas-MuiLink-root bas-MuiTopBreadcrumbsComponent-clickable bas-MuiLink-underlineHover bas-MuiTypography-colorPrimary"})
        models = soup.find_all("a", attrs={'aria-label': 'model'})
        variants = soup.find_all("a", attrs={'aria-label': 'variant'})
        prices = soup.find_all("span", {"class": "bas-MuiCarPriceComponent-value"})
        addresses = soup.find_all("a", {"class": "bas-MuiLink-underlineNone"})
        descriptions = soup.find_all("div", {"class": "bas-MuiAdDescriptionComponent-descriptionText"})
        details = soup.find_all("tr", {"class": "bas-MuiTableRow-root bas-MuiCarFactsComponent-tableRowRoot facts-shown"})
        details2 = soup.find_all("tr", {"class": "bas-MuiTableRow-root bas-MuiCarFactsComponent-tableRowRoot facts-collapsed"})
        general_infos = soup.find_all("tr", {"class": "bas-MuiCarModelInformationComponent-tableRowRoot"})
        equipments = soup.find_all("tr", {"class": "equipment-shown"})

       
        # Udtruk og manipuler data til et dictonary
        for brand, model,variant, price, address, description in zip(brands, models, variants, prices, addresses, descriptions ):

            cars.append({
                "id": urls,
                "brand": brand.text.strip(),
                "model": model.text.strip(),
                "variant": variant.text.strip(),
                "price": price.text.strip(),
                "address": address.text.strip(),
                "description": description.text.strip(),
            })

        for detail in details:
            th = detail.find("th")
            td = detail.find("td")
            if th and td:
                 cars.append({
                    "id": urls,
                     th.text: td.text.strip()})
                 
        for detail2 in details2:
            th = detail2.find("th")
            td = detail2.find("td")
            if th and td:
                 cars.append({
                    "id": urls,
                     th.text: td.text.strip()})
                 
        for general_info in general_infos:
            th = general_info.find("th")
            td = general_info.find("td")
            if th and td:
                 cars.append({
                    "id": urls,
                     th.text: td.text.strip()})
                 
        for equipment in equipments:
            th = equipment.find("th")
            td = equipment.find("td")
            if th and td:
                 cars.append({
                    "id": urls,
                     th.text: th.text.strip(),
                     td.text: td.text.strip()})

scraping_data()


In [385]:
combined_data = {}

for item in cars:
    if item['id'] not in combined_data:
        combined_data[item['id']] = item
    else:
        combined_data[item['id']].update(item)

result = list(combined_data.values())

# make it a dataframe
df = pd.DataFrame(result)
df


,id,brand,model,variant,price,address,description,Modelår,1. registrering,Kilometertal,Drivmiddel,Brændstofforbrug,CO2 udledning,Euronorm,Periodisk afgift,Ydelse,Acceleration,Tophastighed,Geartype,Antal gear,Trækvægt,Farve,Nypris,Kategori,Type,Vægt,Bredde,Længde,Højde,Lasteevne,Max. trækvægt m/bremse,Trækhjul,Cylindre,ABS-bremser,ESP,Airbags,Tankkapacitet,Døre,Fartpilot,Aircondition,Højdejust. førersæde,Alufælge,Læderrat,Antispin,Navigation,AUX tilslutning,Parkeringssensor (bag),Bluetooth,Service ok,Isofix,Musikstreaming via bluetooth,Armlæn,Sædevarme,Servo,Rækkevidde,Batterikapacitet,Energiforbrug,"Elruder, 4x",Elsidespejle,Android auto,Elsidespejle m. varme,Infocenter,Apple carplay,Automatgear,Kørecomputer,"Sædebetræk, stof",Produceret,Centrallås fjernb.,Splitbagsæde,Elruder,Startspærre,LED Kørelys,Automatisk lys,Anhængertræk,Bakkamera,Auto. start/stop,Alarm,Tågelygter,Elsæder,Tagræling,USB tilslutning,Klimaanlæg,Auto. nødbremse,Kunstlæder,Kurvelys,Brugtbilsattest,Centrallås,Parkeringssensor (for),Nøglefri betjening,DAB radio,Regnsensor,Parkeringssensor,Højdejust. forsæde,"Anhængertræk, aftagl.",DAB+ Radio,"Sædebetræk, dellæder",Fuld LED forlygter,Dæktryksmåler,3 individuelle sæder i bag,LED forlygter,Tonede ruder,Digitalt cockpit,"Sædebetræk, læder",CD afspiller,Lygtevasker
0,https://www.bilbasen.dk/brugt/bil/dacia/logan/...,Dacia,Logan,"1,5 dCi 90 Laureate MCV 5d",69.900 kr.,"Vestre Landevej 95, 4951 Nørreballe",Bilen har et stort og rummeligt bagagerum med ...,2014,9/2014,140.000 km,Diesel,"(NEDC) 26,3 km/l",99 g/km,5,2.640 kr. / år,90 hk/220 nm,"12,1 sek.",173 km/t,Manuel,5,1.150 kg,Sort,141.990 kr.,Personbil,St.car,1.090 kg,172 cm,449 cm,155 cm,580 kg,1.150 kg,Forhjulstræk,4,ABS-bremser,Ja,4,50 l,5,Fartpilot,Aircondition,Højdejust. førersæde,Alufælge,Læderrat,Antispin,Navigation,AUX tilslutning,Parkeringssensor (bag),Bluetooth,Service ok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.bilbasen.dk/brugt/bil/dacia/logan/...,Dacia,Logan,"0,9 TCe 90 Family MCV 5d",84.900 kr.,"Tåstruphøj 8, 4300 Holbæk","aut., alu., airc., fjernb. c.lås, fartpilot, u...",2016,10/2016,90.000 km,Benzin,"(NEDC) 20,4 km/l",110 g/km,6,740 kr. / år,90 hk/135 nm,"11,1 sek.",175 km/t,Manuel,5,1.110 kg,-,131.200 kr.,Personbil,St.car,1.038 kg,172 cm,450 cm,155 cm,582 kg,1.110 kg,Forhjulstræk,3,ABS-bremser,Ja,4,50 l,5,Fartpilot,Aircondition,NaN,Alufælge,NaN,Antispin,Navigation,AUX tilslutning,NaN,NaN,NaN,Isofix,Musikstreaming via bluetooth,Armlæn,Sædevarme,Servo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.bilbasen.dk/brugt/bil/dacia/spring...,Dacia,Spring,Comfort+ 5d,139.900 kr.,"Århusvej 115, 8960 Randers SØ","C.lås, Fjernb. c.lås, Parkeringssensor, Parker...",2021,12/2021,2.500 km,El,NaN,NaN,NaN,740 kr. / år,44 hk/125 nm,"19,1 sek.",125 km/t,NaN,NaN,-,Sølvmetal,138.310 kr.,Personbil,SUV,912 kg,162 cm,373 cm,148 cm,388 kg,-,-,NaN,ABS-bremser,Ja,6,NaN,5,NaN,Aircondition,NaN,NaN,NaN,Antispin,NaN,NaN,NaN,NaN,NaN,Isofix,NaN,NaN,NaN,NaN,(WLTP) 230 km,"27,6 kWh",139 Wh/km,"Elruder, 4x",Elsidespejle,Android auto,Elsidespejle m. varme,Infocenter,Apple carplay,Automatgear,Kørecomputer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.bilbasen.dk/brugt/bil/dacia/jogger...,Dacia,Jogger,"1,0 TCe 110 Essential 7prs 5d",203.400 kr.,"Kirstinehøj 62, 2770 Kastrup","💰Danmarks billigste faste rente på KUN 2,99%. ...",2022,10/2022,3.000 km,Benzin,"(NEDC) 21,2 km/l",129 g/km,-,1.200 kr. / år,110 hk/200 nm,"11,2 sek.",180 km/t,Manuel,6,1.200 kg,Hvid,195.810 kr.,Personbil,MPV,1.158 kg,178 cm,45

In [389]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Columns: 105 entries, id to Lygtevasker
dtypes: object(105)
memory usage: 269.2+ KB


In [390]:
df.describe()

,id,brand,model,variant,price,address,description,Modelår,1. registrering,Kilometertal,Drivmiddel,Brændstofforbrug,CO2 udledning,Euronorm,Periodisk afgift,Ydelse,Acceleration,Tophastighed,Geartype,Antal gear,Trækvægt,Farve,Nypris,Kategori,Type,Vægt,Bredde,Længde,Højde,Lasteevne,Max. trækvægt m/bremse,Trækhjul,Cylindre,ABS-bremser,ESP,Airbags,Tankkapacitet,Døre,Fartpilot,Aircondition,Højdejust. førersæde,Alufælge,Læderrat,Antispin,Navigation,AUX tilslutning,Parkeringssensor (bag),Bluetooth,Service ok,Isofix,Musikstreaming via bluetooth,Armlæn,Sædevarme,Servo,Rækkevidde,Batterikapacitet,Energiforbrug,"Elruder, 4x",Elsidespejle,Android auto,Elsidespejle m. varme,Infocenter,Apple carplay,Automatgear,Kørecomputer,"Sædebetræk, stof",Produceret,Centrallås fjernb.,Splitbagsæde,Elruder,Startspærre,LED Kørelys,Automatisk lys,Anhængertræk,Bakkamera,Auto. start/stop,Alarm,Tågelygter,Elsæder,Tagræling,USB tilslutning,Klimaanlæg,Auto. nødbremse,Kunstlæder,Kurvelys,Brugtbilsattest,Centrallås,Parkeringssensor (for),Nøglefri betjening,DAB radio,Regnsensor,Parkeringssensor,Højdejust. forsæde,"Anhængertræk, aftagl.",DAB+ Radio,"Sædebetræk, dellæder",Fuld LED forlygter,Dæktryksmåler,3 individuelle sæder i bag,LED forlygter,Tonede ruder,Digitalt cockpit,"Sædebetræk, læder",CD afspiller,Lygtevasker
count,328,327,327,327,327,327,327,328,328,328,328,287,287,287,328,328,328,328,287,287,328,328,328,328,328,328,328,328,328,328,328,328,287,328,328,328,287,328,62,236,55,108,106,202,118,77,108,91,52,180,95,61,99,88,41,41,41,11,35,50,7,82,50,47,150,99,7,77,40,49,24,64,22,105,37,53,14,19,1,6,12,26,15,12,3,2,19,3,6,3,4,5,6,7,6,1,7,8,2,1,2,1,1,2,1
unique,328,1,9,73,162,167,318,11,95,183,3,43,32,4,20,24,23,23,2,3,12,31,132,1,5,47,11,12,10,50,12,3,2,2,1,3,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
top,https://www.bilbasen.dk/brugt/bil/dacia/logan/...,Dacia,Duster,Comfort+ 5d,99.900 kr.,"Assensvej 1, 5672 Broby",Bliv godt kørende i vores sprit nye Dacia Jogg...,2022,-/-,(ny bil),Benzin,"(NEDC) 26,3 km/l",-,-,740 kr. / år,90 hk/160 nm,"11,1 sek.",169 km/t,Manuel,5,1.100 kg,Sølvmetal,160.810 kr.,Personbil,Halvkombi,912 kg,172 cm,450 cm,155 cm,388 kg,1.100 kg,Forhjulstræk,4,ABS-bremser,Ja,4,50 l,5,Fartpilot,Aircondition,Højdejust. førersæde,Alufælge,Læderrat,Antispin,Navigation,AUX tilslutning,Parkeringssensor (bag),Bluetooth,Service ok,Isofix,Musikstreaming via bluetooth,Armlæn,Sædevarme,Servo,(WLTP) 230 km,"27,6 kWh",139 Wh/km,"Elruder, 4x",Elsidespejle,Android auto,Elsidespejle m. varme,Infocenter,Apple carplay,Automatgear,Kørecomputer,"Sædebetræk, stof",2/2019,Centrallås fjernb.,Splitbagsæde,Elruder,Startspærre,LED Kørelys,Automatisk lys,Anhængertræk,Bakkamera,Auto. start/stop,Alarm,Tågelygter,Elsæder,Tagræling,USB tilslutning,Klimaanlæg,Auto. nødbremse,Kunstlæder,Kurvelys,Brugtbilsattest,Centrallås,Parkeringssensor (for),Nøglefri betjening,DAB radio,Regnsensor,Parkeringssensor,Højdejust. forsæde,"Anhængertræk, aftagl.",DAB+ Radio,"Sædebetræk, dellæder",Fuld LED forlygter,Dæktryksmåler,3 individuelle sæder i bag,LED forlygter,Tonede ruder,Digitalt cockpit,"Sædebetræk, læder",CD afspiller,Lygtevasker
freq,1,327,61,28,20,20,2,75,39,30,173,35,110,194,55,49,43,48,273,177,72,57,17,328,96,41,56,77,56,41,72,283,156,291,328,240,271,328,62,236,55,108,106,202,118,77,108,91,52,180,95,61,99,88,41,39,41,11,35,50,7,82,50,47,150,99,1,77,40,49,24,64,22,105,37,53,14,19,1,6,12,26,15,12,3,2,19,3,6,3,4,5,6,7,6,1,7,8,2,1,2,1,1,2,1


In [386]:
# make the df a excel file
df.to_excel("cars.xlsx")
# make a csv
df.to_csv("cars.csv")


In [391]:
df




,id,brand,model,variant,price,address,description,Modelår,1. registrering,Kilometertal,Drivmiddel,Brændstofforbrug,CO2 udledning,Euronorm,Periodisk afgift,Ydelse,Acceleration,Tophastighed,Geartype,Antal gear,Trækvægt,Farve,Nypris,Kategori,Type,Vægt,Bredde,Længde,Højde,Lasteevne,Max. trækvægt m/bremse,Trækhjul,Cylindre,ABS-bremser,ESP,Airbags,Tankkapacitet,Døre,Fartpilot,Aircondition,Højdejust. førersæde,Alufælge,Læderrat,Antispin,Navigation,AUX tilslutning,Parkeringssensor (bag),Bluetooth,Service ok,Isofix,Musikstreaming via bluetooth,Armlæn,Sædevarme,Servo,Rækkevidde,Batterikapacitet,Energiforbrug,"Elruder, 4x",Elsidespejle,Android auto,Elsidespejle m. varme,Infocenter,Apple carplay,Automatgear,Kørecomputer,"Sædebetræk, stof",Produceret,Centrallås fjernb.,Splitbagsæde,Elruder,Startspærre,LED Kørelys,Automatisk lys,Anhængertræk,Bakkamera,Auto. start/stop,Alarm,Tågelygter,Elsæder,Tagræling,USB tilslutning,Klimaanlæg,Auto. nødbremse,Kunstlæder,Kurvelys,Brugtbilsattest,Centrallås,Parkeringssensor (for),Nøglefri betjening,DAB radio,Regnsensor,Parkeringssensor,Højdejust. forsæde,"Anhængertræk, aftagl.",DAB+ Radio,"Sædebetræk, dellæder",Fuld LED forlygter,Dæktryksmåler,3 individuelle sæder i bag,LED forlygter,Tonede ruder,Digitalt cockpit,"Sædebetræk, læder",CD afspiller,Lygtevasker
0,https://www.bilbasen.dk/brugt/bil/dacia/logan/...,Dacia,Logan,"1,5 dCi 90 Laureate MCV 5d",69.900 kr.,"Vestre Landevej 95, 4951 Nørreballe",Bilen har et stort og rummeligt bagagerum med ...,2014,9/2014,140.000 km,Diesel,"(NEDC) 26,3 km/l",99 g/km,5,2.640 kr. / år,90 hk/220 nm,"12,1 sek.",173 km/t,Manuel,5,1.150 kg,Sort,141.990 kr.,Personbil,St.car,1.090 kg,172 cm,449 cm,155 cm,580 kg,1.150 kg,Forhjulstræk,4,ABS-bremser,Ja,4,50 l,5,Fartpilot,Aircondition,Højdejust. førersæde,Alufælge,Læderrat,Antispin,Navigation,AUX tilslutning,Parkeringssensor (bag),Bluetooth,Service ok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.bilbasen.dk/brugt/bil/dacia/logan/...,Dacia,Logan,"0,9 TCe 90 Family MCV 5d",84.900 kr.,"Tåstruphøj 8, 4300 Holbæk","aut., alu., airc., fjernb. c.lås, fartpilot, u...",2016,10/2016,90.000 km,Benzin,"(NEDC) 20,4 km/l",110 g/km,6,740 kr. / år,90 hk/135 nm,"11,1 sek.",175 km/t,Manuel,5,1.110 kg,-,131.200 kr.,Personbil,St.car,1.038 kg,172 cm,450 cm,155 cm,582 kg,1.110 kg,Forhjulstræk,3,ABS-bremser,Ja,4,50 l,5,Fartpilot,Aircondition,NaN,Alufælge,NaN,Antispin,Navigation,AUX tilslutning,NaN,NaN,NaN,Isofix,Musikstreaming via bluetooth,Armlæn,Sædevarme,Servo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.bilbasen.dk/brugt/bil/dacia/spring...,Dacia,Spring,Comfort+ 5d,139.900 kr.,"Århusvej 115, 8960 Randers SØ","C.lås, Fjernb. c.lås, Parkeringssensor, Parker...",2021,12/2021,2.500 km,El,NaN,NaN,NaN,740 kr. / år,44 hk/125 nm,"19,1 sek.",125 km/t,NaN,NaN,-,Sølvmetal,138.310 kr.,Personbil,SUV,912 kg,162 cm,373 cm,148 cm,388 kg,-,-,NaN,ABS-bremser,Ja,6,NaN,5,NaN,Aircondition,NaN,NaN,NaN,Antispin,NaN,NaN,NaN,NaN,NaN,Isofix,NaN,NaN,NaN,NaN,(WLTP) 230 km,"27,6 kWh",139 Wh/km,"Elruder, 4x",Elsidespejle,Android auto,Elsidespejle m. varme,Infocenter,Apple carplay,Automatgear,Kørecomputer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.bilbasen.dk/brugt/bil/dacia/jogger...,Dacia,Jogger,"1,0 TCe 110 Essential 7prs 5d",203.400 kr.,"Kirstinehøj 62, 2770 Kastrup","💰Danmarks billigste faste rente på KUN 2,99%. ...",2022,10/2022,3.000 km,Benzin,"(NEDC) 21,2 km/l",129 g/km,-,1.200 kr. / år,110 hk/200 nm,"11,2 sek.",180 km/t,Manuel,6,1.200 kg,Hvid,195.810 kr.,Personbil,MPV,1.158 kg,178 cm,45

In [393]:
# make a new df from

















